In [1]:
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

def custom_preprocessing(img):
    # Convert the image from float [0,1] to uint8 [0,255]
    img = (img * 255).astype(np.uint8)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    
    # Highlight edges
    highlighted = cv2.subtract(img, blurred)
    highlighted_plus = cv2.add(img, highlighted)
    
    # Apply Laplacian high-pass filter
    final_img = cv2.Laplacian(highlighted_plus, cv2.CV_8U)
    # final_img = cv2.add(img, laplacian)

    # Convert the image back to float [0,1]
    final_img = final_img.astype(np.float32) / 255.0
    
    # Ensure values are between 0 and 1
    final_img = np.clip(final_img, 0, 1)
    
    return final_img


train_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing
)




In [3]:
import os

def count_images_in_dir(directory):
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff']
    count = 0

    for subdir, _, files in os.walk(directory):
        for file in files:
            if os.path.splitext(file)[1].lower() in image_extensions:
                count += 1

    return count

# Example usage:
train_dir = '../train/'
test_dir = '../test/'

print(f"Number of images in {train_dir}: {count_images_in_dir(train_dir)}")
print(f"Number of images in {test_dir}: {count_images_in_dir(test_dir)}")


Number of images in ../train/: 3507
Number of images in ../test/: 1796


In [4]:
# Directory paths
# train_dir = 'D:\\study\\CS4243\\project\\img\\train'
train_dir = '../train/'
test_dir = '../test/'

# Data preprocessing
# train_datagen = ImageDataGenerator(rescale=1./255,
#                                    rotation_range=40,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True,
#                                    fill_mode='nearest')

# test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)






Found 3507 images belonging to 2 classes.
Found 1796 images belonging to 2 classes.


In [4]:
from keras.regularizers import l2


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.5),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])





In [5]:
from keras.regularizers import l2
#simpler model


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3), kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.4),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)), # reduced neurons
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [15]:
from keras.regularizers import l2

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)), #, kernel_regularizer=l2(0.0005)),  # Reduced regularization strength
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # kernel_regularizer=l2(0.0005)),  # Reduced regularization strength
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=l2(0.0005)),  # Reduced neurons & regularization strength
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(128, 128, 3)),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
from keras.callbacks import EarlyStopping

# Compiling the model remains the same
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
# Training the model

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=7,
    validation_data= test_generator,
    validation_steps=len(test_generator),
    callbacks=[early_stopping]
)

# Saving the model
model.save('weapon_detection_model.h5')

# Optionally, you could evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Epoch 1/7
110/110 [==============================] - 635s 6s/step - loss: 5.1165 - accuracy: 0.5632 - val_loss: 0.9242 - val_accuracy: 0.7133
Epoch 2/7
110/110 [==============================] - 632s 6s/step - loss: 0.9366 - accuracy: 0.5908 - val_loss: 0.8805 - val_accuracy: 0.6882
Epoch 3/7
110/110 [==============================] - 631s 6s/step - loss: 0.8985 - accuracy: 0.6074 - val_loss: 0.8662 - val_accuracy: 0.6938
Epoch 4/7
110/110 [==============================] - 26820s 246s/step - loss: 0.8668 - accuracy: 0.6088 - val_loss: 0.8496 - val_accuracy: 0.6565
Epoch 5/7
 20/110 [====>.........................] - ETA: 1:58 - loss: 0.8607 - accuracy: 0.6156

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_dir = '../train/'
test_dir = '../test/'

# Data Preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(128, 128),
                                                    batch_size=16,
                                                    class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(128, 128),
                                                  batch_size=16,
                                                  class_mode='binary')

# Model Architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=test_generator)

# Save the Model
model.save('weapon_detection_model.h5')


Found 3507 images belonging to 2 classes.
Found 1796 images belonging to 2 classes.
Epoch 1/10
220/220 [==============================] - 846s 4s/step - loss: 0.6296 - accuracy: 0.6715 - val_loss: 0.5688 - val_accuracy: 0.7261
Epoch 2/10
220/220 [==============================] - 638s 3s/step - loss: 0.5776 - accuracy: 0.7009 - val_loss: 0.6583 - val_accuracy: 0.6186
Epoch 3/10
 20/220 [=>............................] - ETA: 5:57 - loss: 0.5299 - accuracy: 0.7500

ResourceExhaustedError: Graph execution error:

Detected at node Adam/sub_18 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Steven\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Steven\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance

  File "C:\Users\Steven\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Steven\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\Steven\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\Steven\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Users\Steven\AppData\Local\Programs\Python